# এজেন্ট-টু-এজেন্ট (A2A) প্রোটোকল ব্যবহার করে সেম্যান্টিক কের্নেল এবং অ্যাজুর ওপেনএআই

এই নোটবুকটি দেখায় কীভাবে সেম্যান্টিক কের্নেলকে A2A প্রোটোকলের সাথে ব্যবহার করে অ্যাজুর ওপেনএআই এবং অ্যাজুর AI ফাউন্ড্রির মাধ্যমে একটি বহু-এজেন্ট ভ্রমণ পরিকল্পনা ব্যবস্থা তৈরি করা যায়। আপনার পরিবেশ ভেরিয়েবল সেটআপ করতে, আপনি [সেটআপ পাঠ](/00-course-setup/README.md) অনুসরণ করতে পারেন।

## আপনি যা তৈরি করবেন

তিনটি এজেন্টের একটি ভ্রমণ পরিকল্পনা ব্যবস্থা:
1. **কারেন্সি এক্সচেঞ্জ এজেন্ট** - রিয়েল-টাইম বিনিময় হার ব্যবহার করে মুদ্রা রূপান্তর পরিচালনা করে
2. **অ্যাক্টিভিটি প্ল্যানার এজেন্ট** - কার্যক্রম পরিকল্পনা করে এবং ভ্রমণের সুপারিশ প্রদান করে
3. **ট্রাভেল ম্যানেজার এজেন্ট** - অন্যান্য এজেন্টদের সমন্বয় করে ব্যাপক ভ্রমণ সহায়তা প্রদান করে


## ইনস্টলেশন

প্রথমে, প্রয়োজনীয় ডিপেনডেন্সিগুলো ইনস্টল করি:


## প্রয়োজনীয় লাইব্রেরি আমদানি করুন


In [ ]:
import asyncio
import json
import logging
import os
import threading
import time
from typing import Any, Annotated, AsyncIterable, Literal
from enum import Enum

import httpx
import nest_asyncio
import uvicorn
from dotenv import load_dotenv
from pydantic import BaseModel

# A2A imports
# Add this to your imports at the top
from a2a.server.agent_execution import AgentExecutor
from a2a.client import ClientConfig, ClientFactory, create_text_message_object
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore, InMemoryPushNotificationConfigStore, BasePushNotificationSender
from a2a.types import (
    AgentCapabilities,
    AgentCard,
    AgentSkill,
    TransportProtocol,
)
from a2a.utils.constants import AGENT_CARD_WELL_KNOWN_PATH

# Semantic Kernel imports
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    OpenAIChatPromptExecutionSettings,
)
from semantic_kernel.contents import (
    FunctionCallContent,
    FunctionResultContent,
    StreamingTextContent,
)
from semantic_kernel.functions import KernelArguments, kernel_function

## পরিবেশ কনফিগারেশন

Azure OpenAI সেটিংস কনফিগার করুন। নিশ্চিত করুন যে আপনার নিম্নলিখিত পরিবেশ ভেরিয়েবলগুলি সেট করা আছে:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Load environment variables
load_dotenv()

# Apply nest_asyncio for running async code in Jupyter
nest_asyncio.apply()

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
)
logger = logging.getLogger(__name__)

print('Environment configured successfully!')


## মুদ্রা প্লাগইন সংজ্ঞায়িত করুন

এই প্লাগইনটি Frankfurter API ব্যবহার করে রিয়েল-টাইম মুদ্রা বিনিময় হার প্রদান করে।


In [ ]:
class CurrencyPlugin:
    """A currency plugin that leverages Frankfurter API for exchange rates."""

    @kernel_function(
        description='Retrieves exchange rate between currency_from and currency_to using Frankfurter API'
    )
    def get_exchange_rate(
        self,
        currency_from: Annotated[str, 'Currency code to convert from, e.g. USD'],
        currency_to: Annotated[str, 'Currency code to convert to, e.g. EUR or INR'],
        date: Annotated[str, "Date or 'latest'"] = 'latest',
    ) -> str:
        try:
            response = httpx.get(
                f'https://api.frankfurter.app/{date}',
                params={'from': currency_from, 'to': currency_to},
                timeout=10.0,
            )
            response.raise_for_status()
            data = response.json()
            if 'rates' not in data or currency_to not in data['rates']:
                return f'Could not retrieve rate for {currency_from} to {currency_to}'
            rate = data['rates'][currency_to]
            return f'1 {currency_from} = {rate} {currency_to}'
        except Exception as e:
            return f'Currency API call failed: {str(e)}'

print('✅ Currency Plugin defined')

## প্রতিক্রিয়া বিন্যাস নির্ধারণ করুন

এজেন্ট আউটপুটের জন্য কাঠামোবদ্ধ প্রতিক্রিয়া বিন্যাস।


In [ ]:
class ResponseFormat(BaseModel):
    """A Response Format model to direct how the model should respond."""
    status: Literal['input_required', 'completed', 'error'] = 'input_required'
    message: str

print('✅ Response format defined')

## A2A এজেন্ট এক্সিকিউটর তৈরি করুন

এটি সেমান্টিক কোর এজেন্টগুলোকে A2A প্রোটোকলের সাথে কাজ করার জন্য মোড়ক দেয়।


In [ ]:
class SemanticKernelTravelAgentExecutor(AgentExecutor):
    """A2A Executor for Semantic Kernel Travel Agent."""

    def __init__(self):
        # Create Azure OpenAI service
        self.chat_service = AzureChatCompletion(
            deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
            endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        )

        # Create Currency Exchange Agent
        self.currency_agent = ChatCompletionAgent(
            service=self.chat_service,
            name='CurrencyExchangeAgent',
            instructions=(
                'You specialize in handling currency-related requests from travelers. '
                'This includes providing current exchange rates, converting amounts between different currencies, '
                'explaining fees or charges related to currency exchange, and giving advice on the best practices for exchanging currency. '
                'Your goal is to assist travelers promptly and accurately with all currency-related questions.'
            ),
            plugins=[CurrencyPlugin()],
        )

        # Create Activity Planner Agent
        self.activity_agent = ChatCompletionAgent(
            service=self.chat_service,
            name='ActivityPlannerAgent',
            instructions=(
                'You specialize in planning and recommending activities for travelers. '
                'This includes suggesting sightseeing options, local events, dining recommendations, '
                'booking tickets for attractions, advising on travel itineraries, and ensuring activities '
                'align with traveler preferences and schedule. '
                'Your goal is to create enjoyable and personalized experiences for travelers.'
            ),
        )

        # Create the main Travel Manager Agent - simplified approach
        self.travel_agent = ChatCompletionAgent(
            service=self.chat_service,
            name='TravelManagerAgent',
            instructions=(
                "Your role is to carefully analyze the traveler's request and forward it to the appropriate agent based on the "
                'specific details of the query. '
                'Forward any requests involving monetary amounts, currency exchange rates, currency conversions, fees related '
                'to currency exchange, financial transactions, or payment methods to the CurrencyExchangeAgent. '
                'Forward requests related to planning activities, sightseeing recommendations, dining suggestions, event '
                'booking, itinerary creation, or any experiential aspects of travel that do not explicitly involve monetary '
                'transactions to the ActivityPlannerAgent. '
                'Your primary goal is precise and efficient delegation to ensure travelers receive accurate and specialized '
                'assistance promptly.'
            ),
            plugins=[self.currency_agent, self.activity_agent],
        )

        self.thread = None
        self.SUPPORTED_CONTENT_TYPES = ['text', 'text/plain']

    async def execute(self, context, event_queue):
        """Execute method required by A2A framework."""
        try:
            # Import required A2A utilities
            from a2a.utils import new_agent_text_message, new_task, new_text_artifact
            from a2a.types import TaskArtifactUpdateEvent, TaskState, TaskStatus, TaskStatusUpdateEvent

            # Get user input using the correct context method
            user_input = context.get_user_input()
            task = context.current_task

            if not task:
                task = new_task(context.message)
                await event_queue.enqueue_event(task)

            # Ensure thread exists
            session_id = task.context_id
            await self._ensure_thread_exists(session_id)

            # Process the request - no special response format, let it respond naturally
            response = await self.travel_agent.get_response(
                messages=user_input,
                thread=self.thread,
            )

            # Get the content directly as string
            content = response.content if isinstance(response.content, str) else str(response.content)

            # Send completion event with artifact
            await event_queue.enqueue_event(
                TaskArtifactUpdateEvent(
                    append=False,
                    context_id=task.context_id,
                    task_id=task.id,
                    last_chunk=True,
                    artifact=new_text_artifact(
                        name='travel_result',
                        description='Travel planning result',
                        text=content,
                    ),
                )
            )
            
            await event_queue.enqueue_event(
                TaskStatusUpdateEvent(
                    status=TaskStatus(state=TaskState.completed),
                    final=True,
                    context_id=task.context_id,
                    task_id=task.id,
                )
            )

        except Exception as e:
            logger.error(f"Error in SemanticKernelTravelAgentExecutor.execute: {str(e)}")
            # Send error status
            await event_queue.enqueue_event(
                TaskStatusUpdateEvent(
                    status=TaskStatus(
                        state=TaskState.input_required,
                        message=new_agent_text_message(
                            f"Error processing request: {str(e)}",
                            task.context_id,
                            task.id,
                        ),
                    ),
                    final=True,
                    context_id=task.context_id,
                    task_id=task.id,
                )
            )

    async def cancel(self, context, event_queue):
        """Cancel method - not supported for this agent."""
        raise Exception('cancel not supported')

    async def _ensure_thread_exists(self, session_id: str) -> None:
        """Ensure thread exists for the session."""
        if self.thread is None or self.thread.id != session_id:
            if self.thread:
                await self.thread.delete()
            self.thread = ChatHistoryAgentThread(thread_id=session_id)


print('✅ Travel Manager Agent Executor simplified - removed JSON formatting constraints')

## পৃথক A2A এজেন্ট তৈরি করুন

এখন আমরা প্রতিটি বিশেষায়িত এজেন্টের জন্য A2A র‍্যাপার তৈরি করব।


In [ ]:
class CurrencyAgentExecutor(AgentExecutor):
    """A2A Executor for Currency Exchange Agent."""

    def __init__(self):
        self.chat_service = AzureChatCompletion(
            deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
            endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        )

        self.agent = ChatCompletionAgent(
            service=self.chat_service,
            name='CurrencyExchangeAgent',
            instructions=(
                'You are a currency exchange specialist. Provide accurate exchange rates and currency conversion information. '
                'Use the get_exchange_rate function to get real-time rates. '
                'Always provide clear, concise information about currency conversions.'
            ),
            plugins=[CurrencyPlugin()],
        )
        self.thread = None
        self.SUPPORTED_CONTENT_TYPES = ['text', 'text/plain']

    async def execute(self, context, event_queue):
        """Execute method required by A2A framework."""
        try:
            # Import required A2A utilities
            from a2a.utils import new_agent_text_message, new_task, new_text_artifact
            from a2a.types import TaskArtifactUpdateEvent, TaskState, TaskStatus, TaskStatusUpdateEvent

            # Get user input using the correct context method
            user_input = context.get_user_input()
            task = context.current_task
            
            if not task:
                task = new_task(context.message)
                await event_queue.enqueue_event(task)

            # Ensure thread exists
            session_id = task.context_id
            if self.thread is None or self.thread.id != session_id:
                if self.thread:
                    await self.thread.delete()
                self.thread = ChatHistoryAgentThread(thread_id=session_id)

            # Process the request
            response = await self.agent.get_response(messages=user_input, thread=self.thread)
            content = response.content if isinstance(response.content, str) else str(response.content)

            # Send completion event with artifact
            await event_queue.enqueue_event(
                TaskArtifactUpdateEvent(
                    append=False,
                    context_id=task.context_id,
                    task_id=task.id,
                    last_chunk=True,
                    artifact=new_text_artifact(
                        name='currency_result',
                        description='Currency exchange information',
                        text=content,
                    ),
                )
            )
            
            await event_queue.enqueue_event(
                TaskStatusUpdateEvent(
                    status=TaskStatus(state=TaskState.completed),
                    final=True,
                    context_id=task.context_id,
                    task_id=task.id,
                )
            )

        except Exception as e:
            logger.error(f"Error in CurrencyAgentExecutor.execute: {str(e)}")
            # Send error status
            await event_queue.enqueue_event(
                TaskStatusUpdateEvent(
                    status=TaskStatus(
                        state=TaskState.input_required,
                        message=new_agent_text_message(
                            f"Error processing request: {str(e)}",
                            task.context_id,
                            task.id,
                        ),
                    ),
                    final=True,
                    context_id=task.context_id,
                    task_id=task.id,
                )
            )

    async def cancel(self, context, event_queue):
        """Cancel method - not supported for this simple agent."""
        raise Exception('cancel not supported')

# Updated Activity Planner Agent Executor
class ActivityAgentExecutor(AgentExecutor):
    """A2A Executor for Activity Planner Agent."""

    def __init__(self):
        self.chat_service = AzureChatCompletion(
            deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
            endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        )

        self.agent = ChatCompletionAgent(
            service=self.chat_service,
            name='ActivityPlannerAgent',
            instructions=(
                'You are a travel activity planning specialist. Create detailed, personalized activity recommendations. '
                'Include specific times, locations, and practical tips. '
                'Consider budget, preferences, and local culture in your suggestions.'
            ),
        )
        self.thread = None
        self.SUPPORTED_CONTENT_TYPES = ['text', 'text/plain']

    async def execute(self, context, event_queue):
        """Execute method required by A2A framework."""
        try:
            # Import required A2A utilities
            from a2a.utils import new_agent_text_message, new_task, new_text_artifact
            from a2a.types import TaskArtifactUpdateEvent, TaskState, TaskStatus, TaskStatusUpdateEvent

            # Get user input using the correct context method
            user_input = context.get_user_input()
            task = context.current_task
            
            if not task:
                task = new_task(context.message)
                await event_queue.enqueue_event(task)

            # Ensure thread exists
            session_id = task.context_id
            if self.thread is None or self.thread.id != session_id:
                if self.thread:
                    await self.thread.delete()
                self.thread = ChatHistoryAgentThread(thread_id=session_id)

            # Process the request
            response = await self.agent.get_response(messages=user_input, thread=self.thread)
            content = response.content if isinstance(response.content, str) else str(response.content)

            # Send completion event with artifact
            await event_queue.enqueue_event(
                TaskArtifactUpdateEvent(
                    append=False,
                    context_id=task.context_id,
                    task_id=task.id,
                    last_chunk=True,
                    artifact=new_text_artifact(
                        name='activity_result',
                        description='Activity planning recommendations',
                        text=content,
                    ),
                )
            )
            
            await event_queue.enqueue_event(
                TaskStatusUpdateEvent(
                    status=TaskStatus(state=TaskState.completed),
                    final=True,
                    context_id=task.context_id,
                    task_id=task.id,
                )
            )

        except Exception as e:
            logger.error(f"Error in ActivityAgentExecutor.execute: {str(e)}")
            # Send error status
            await event_queue.enqueue_event(
                TaskStatusUpdateEvent(
                    status=TaskStatus(
                        state=TaskState.input_required,
                        message=new_agent_text_message(
                            f"Error processing request: {str(e)}",
                            task.context_id,
                            task.id,
                        ),
                    ),
                    final=True,
                    context_id=task.context_id,
                    task_id=task.id,
                )
            )

    async def cancel(self, context, event_queue):
        """Cancel method - not supported for this simple agent."""
        raise Exception('cancel not supported')

## এজেন্ট কার্ড সংজ্ঞায়িত করুন

এজেন্ট কার্ডগুলি A2A ডিসকভারি জন্য প্রতিটি এজেন্টের সক্ষমতাগুলি বর্ণনা করে।


In [ ]:
# Currency Exchange Agent Card
currency_agent_card = AgentCard(
    name='Currency Exchange Agent',
    url='http://localhost:10020',
    description='Provides real-time currency exchange rates and conversion services',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['text/plain'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='currency_exchange',
            name='Currency Exchange',
            description='Get exchange rates and convert between currencies',
            tags=['currency', 'exchange', 'conversion', 'forex'],
            examples=[
                'What is the exchange rate from USD to EUR?',
                'Convert 1000 USD to JPY',
                'How much is 500 EUR in GBP?',
            ],
        )
    ],
)

# Activity Planner Agent Card
activity_agent_card = AgentCard(
    name='Activity Planner Agent',
    url='http://localhost:10021',
    description='Plans activities and provides travel recommendations',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['text/plain'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='activity_planning',
            name='Activity Planning',
            description='Create personalized travel itineraries and activity recommendations',
            tags=['travel', 'activities', 'itinerary', 'recommendations'],
            examples=[
                'Plan a day trip in Paris',
                'Recommend restaurants in Tokyo',
                'What are the must-see attractions in Rome?',
            ],
        )
    ],
)

# Travel Manager Agent Card (Main orchestrator)
travel_manager_card = AgentCard(
    name='SK Travel Manager',
    url='http://localhost:10022',
    description='Comprehensive travel planning agent that orchestrates currency and activity services',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['text/plain'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='comprehensive_travel_planning',
            name='Comprehensive Travel Planning',
            description='Handles all aspects of travel planning including currency and activities',
            tags=['travel', 'planning', 'currency', 'activities', 'orchestration'],
            examples=[
                'Plan a budget-friendly trip to Seoul with currency exchange info',
                'I need help with my Tokyo trip including money exchange and activities',
                'What should I do in London and how much money should I exchange?',
            ],
        )
    ],
)

print('✅ Agent cards defined')

## A2A সার্ভার হেল্পার ফাংশন তৈরি করুন


এই ফাংশনটি একটি A2A (Agent-to-Agent) প্রোটোকল সার্ভার তৈরি করে HTTP যোগাযোগ অবকাঠামো স্থাপন করে, টাস্ক ম্যানেজমেন্ট এবং পুশ নোটিফিকেশন সহ একটি রিকোয়েস্ট হ্যান্ডলার কনফিগার করে, সবকিছু Starlette ওয়েব অ্যাপ্লিকেশনে মোড়ানো হয় এবং রানযোগ্য সার্ভার ইনস্ট্যান্সটি ফেরত দেয়।

A2AStarletteApplication হলো একটি ওয়েব অ্যাপ্লিকেশন র‍্যাপার যা Starlette ASGI ফ্রেমওয়ার্কের উপর ভিত্তি করে তৈরি হয়েছে। এটি A2A প্রোটোকল স্ট্যান্ডার্ড বাস্তবায়ন করে, যা AI এজেন্টদের মধ্যে HTTP এর মাধ্যমে স্ট্যান্ডার্ডাইজড মেসেজ ফরম্যাট এবং ডিসকভারি মেকানিজম ব্যবহার করে যোগাযোগের সুযোগ দেয়।


In [ ]:
def create_a2a_server(agent_executor, agent_card):
    """Create an A2A server for any agent executor."""
    httpx_client = httpx.AsyncClient()
    push_config_store = InMemoryPushNotificationConfigStore()

    request_handler = DefaultRequestHandler(
        agent_executor=agent_executor,
        task_store=InMemoryTaskStore(),
        push_config_store=push_config_store,
        push_sender=BasePushNotificationSender(
            httpx_client, push_config_store),
    )

    app = A2AStarletteApplication(
        agent_card=agent_card,
        http_handler=request_handler
    )

    # Return the actual Starlette app
    # Check if we need to build or get the app
    if hasattr(app, 'build'):
        return app.build()
    elif hasattr(app, 'app'):
        return app.app
    else:
        return app


print('✅ A2A server helper function created')

## সমস্ত A2A সার্ভার শুরু করুন

আমরা তিনটি এজেন্টকে আলাদা A2A সার্ভার হিসেবে uvicorn ব্যবহার করে চালাব।


In [ ]:

async def run_agent_server(agent_executor, agent_card, port):
    """Run a single agent server with proper error handling."""
    try:
        app = create_a2a_server(agent_executor, agent_card)

        config = uvicorn.Config(
            app,
            host='127.0.0.1',
            port=port,
            log_level='info',
            loop='none',
            timeout_keep_alive=30,
            limit_concurrency=100,
        )

        server = uvicorn.Server(config)
        await server.serve()
    except Exception as e:
        logger.error(f"Error starting server on port {port}: {str(e)}")
        raise


# ...existing code...

# Global variable to track running servers
running_servers = []


async def start_all_servers_background():
    """Start all servers in background tasks."""
    global running_servers

    try:
        # Create agent executors
        currency_executor = CurrencyAgentExecutor()
        activity_executor = ActivityAgentExecutor()
        travel_executor = SemanticKernelTravelAgentExecutor()

        # Create tasks for all servers
        tasks = [
            asyncio.create_task(run_agent_server(
                currency_executor, currency_agent_card, 10020)),
            asyncio.create_task(run_agent_server(
                activity_executor, activity_agent_card, 10021)),
            asyncio.create_task(run_agent_server(
                travel_executor, travel_manager_card, 10022)),
        ]

        running_servers = tasks

        # Give servers time to start
        await asyncio.sleep(3)

        print('✅ All A2A agent servers started in background!')
        print('   - Currency Exchange Agent: http://127.0.0.1:10020')
        print('   - Activity Planner Agent: http://127.0.0.1:10021')
        print('   - Travel Manager Agent: http://127.0.0.1:10022')

        # Don't await the tasks here - let them run in background
        return tasks

    except Exception as e:
        logger.error(f"Error in start_all_servers: {str(e)}")
        print(f"Failed to start servers: {str(e)}")
        raise

# Start the servers in background
server_tasks = await start_all_servers_background()

In [ ]:
# Add this cell after starting the servers but before testing
async def verify_servers():
    """Verify that all A2A servers are running and accessible."""
    import httpx

    servers = [
        ('Currency Exchange Agent', 'http://localhost:10020'),
        ('Activity Planner Agent', 'http://localhost:10021'),
        ('Travel Manager Agent', 'http://localhost:10022'),
    ]

    print("🔍 Verifying A2A servers...")
    print("="*50)

    async with httpx.AsyncClient() as client:
        for name, url in servers:
            try:
                response = await client.get(f"{url}{AGENT_CARD_WELL_KNOWN_PATH}", timeout=5.0)
                if response.status_code == 200:
                    print(f"✅ {name} is running at {url}")
                else:
                    print(f"⚠️ {name} returned status {response.status_code}")
            except Exception as e:
                print(f"❌ {name} is not accessible: {str(e)}")

    print("="*50)

# Run server verification
await verify_servers()



## A2A ক্লায়েন্ট তৈরি করুন

এখন চলুন আমাদের A2A এজেন্টদের সাথে যোগাযোগ করার জন্য একটি ক্লায়েন্ট তৈরি করি।


In [ ]:
class A2AClient:
    """Simple A2A client to interact with A2A servers."""
    
    def __init__(self, default_timeout: float = 60.0):
        self._agent_info_cache = {}
        self.default_timeout = default_timeout
    
    async def send_message(self, agent_url: str, message: str) -> str:
        """Send a message to an A2A agent."""
        timeout_config = httpx.Timeout(
            timeout=self.default_timeout,
            connect=10.0,
            read=self.default_timeout,
            write=10.0,
            pool=5.0,
        )
        
        async with httpx.AsyncClient(timeout=timeout_config) as httpx_client:
            # Fetch agent card if not cached
            if agent_url not in self._agent_info_cache:
                agent_card_response = await httpx_client.get(
                    f'{agent_url}{AGENT_CARD_WELL_KNOWN_PATH}'
                )
                self._agent_info_cache[agent_url] = agent_card_response.json()
            
            agent_card_data = self._agent_info_cache[agent_url]
            agent_card = AgentCard(**agent_card_data)
            
            # Create A2A client
            config = ClientConfig(
                httpx_client=httpx_client,
                supported_transports=[
                    TransportProtocol.jsonrpc,
                    TransportProtocol.http_json,
                ],
                use_client_preference=True,
            )
            
            factory = ClientFactory(config)
            client = factory.create(agent_card)
            
            # Create and send message
            message_obj = create_text_message_object(content=message)
            
            responses = []
            async for response in client.send_message(message_obj):
                responses.append(response)
            
            # Enhanced response parsing
            if responses:
                try:
                    # The response should be a tuple (task, any_additional_data)
                    for response_item in responses:
                        if isinstance(response_item, tuple) and len(response_item) > 0:
                            task = response_item[0]
                            
                            # Try multiple ways to extract the response text
                            if hasattr(task, 'artifacts') and task.artifacts:
                                for artifact in task.artifacts:
                                    if hasattr(artifact, 'parts') and artifact.parts:
                                        for part in artifact.parts:
                                            if hasattr(part, 'root') and hasattr(part.root, 'text'):
                                                return part.root.text
                                            elif hasattr(part, 'text'):
                                                return part.text
                                            elif hasattr(part, 'content'):
                                                return str(part.content)
                                    elif hasattr(artifact, 'text'):
                                        return artifact.text
                                    elif hasattr(artifact, 'content'):
                                        return str(artifact.content)
                            
                            # If artifacts don't work, try direct task properties
                            elif hasattr(task, 'text'):
                                return task.text
                            elif hasattr(task, 'content'):
                                return str(task.content)
                            elif hasattr(task, 'result'):
                                return str(task.result)
                            else:
                                # Debug: print the task structure
                                print(f"Debug - Task type: {type(task)}")
                                print(f"Debug - Task attributes: {dir(task)}")
                                if hasattr(task, '__dict__'):
                                    print(f"Debug - Task dict: {task.__dict__}")
                                return f"Received response but couldn't parse content. Task: {str(task)}"
                        else:
                            # Handle direct response objects
                            response_text = str(response_item)
                            if response_text and response_text != "None":
                                return response_text
                    
                    return f"Received {len(responses)} responses but couldn't extract text content"
                    
                except Exception as e:
                    return f"Error parsing response: {str(e)}. Raw responses: {str(responses)}"
            
            return 'No response received'

# Create client instance
a2a_client = A2AClient()
print('✅ A2A client updated with enhanced response parsing')

## পৃথক এজেন্ট পরীক্ষা করুন

প্রতিটি এজেন্টকে আলাদাভাবে পরীক্ষা করে দেখি তারা কীভাবে কাজ করে।


In [ ]:
# Test Currency Exchange Agent
async def test_currency_agent():
    """Test the currency exchange agent."""
    print("\n🔍 Testing Currency Exchange Agent")
    print("="*50)
    
    response = await a2a_client.send_message(
        'http://localhost:10020',
        'What is the exchange rate from USD to EUR and JPY?'
    )
    
    print("User: What is the exchange rate from USD to EUR and JPY?")
    print("\nCurrency Agent Response:")
    print(response)

await test_currency_agent()

In [ ]:
# Test Activity Planner Agent
async def test_activity_agent():
    """Test the activity planner agent."""
    print("\n🔍 Testing Activity Planner Agent")
    print("="*50)
    
    response = await a2a_client.send_message(
        'http://localhost:10021',
        'Plan a one-day itinerary for Paris including must-see attractions'
    )
    
    print("User: Plan a one-day itinerary for Paris including must-see attractions")
    print("\nActivity Agent Response:")
    print(response)

await test_activity_agent()

## ভ্রমণ ব্যবস্থাপক (অর্কেস্ট্রেটর) পরীক্ষা করুন

এখন চলুন প্রধান ভ্রমণ ব্যবস্থাপক এজেন্টটি পরীক্ষা করি, যা অন্যান্য এজেন্টদের পরিচালনা করে।


In [ ]:
# Test Travel Manager with comprehensive request
async def test_travel_manager():
    """Test the travel manager orchestrating multiple agents."""
    print("\n🔍 Testing Travel Manager Agent (Orchestrator)")
    print("="*50)
    
    response = await a2a_client.send_message(
        'http://localhost:10022',
        'I am planning a trip to Tokyo. I have 1000 USD to exchange. What is the current exchange rate to JPY and what activities do you recommend for a 2-day visit?'
    )
    
    print("User: I am planning a trip to Tokyo. I have 1000 USD to exchange.")
    print("      What is the current exchange rate to JPY and what activities")
    print("      do you recommend for a 2-day visit?")
    print("\nTravel Manager Response:")
    print(response)

await test_travel_manager()

## ইন্টারঅ্যাকটিভ টেস্টিং

আপনার নিজের ভ্রমণ পরিকল্পনার প্রশ্নগুলি চেষ্টা করুন!


In [ ]:
async def interactive_test():
    """Interactive testing function."""
    print("\n🎯 Interactive Travel Planning Assistant")
    print("="*50)
    print("Available agents:")
    print("1. Currency Exchange Agent (port 10020) - Currency conversions")
    print("2. Activity Planner Agent (port 10021) - Travel recommendations")
    print("3. Travel Manager Agent (port 10022) - Comprehensive planning")
    print("\nExample queries:")
    print("- 'Convert 500 EUR to GBP'")
    print("- 'Plan a romantic dinner in Rome'")
    print("- 'I need help planning a budget trip to Seoul with 2000 USD'")
    
    # You can modify this query to test different scenarios
    user_query = "I'm visiting London next week with a budget of 1500 EUR. What's the exchange rate to GBP and what are the top attractions I should visit?"
    
    print(f"\nYour query: {user_query}")
    print("\nProcessing with Travel Manager...")
    
    response = await a2a_client.send_message(
        'http://localhost:10022',
        user_query
    )
    
    print("\nResponse:")
    print(response)

await interactive_test()

## সারসংক্ষেপ

অভিনন্দন! আপনি সফলভাবে একটি মাল্টি-এজেন্ট ট্রাভেল প্ল্যানিং সিস্টেম তৈরি করেছেন, যেখানে ব্যবহার করা হয়েছে:

### ব্যবহৃত প্রযুক্তি:
- **Semantic Kernel** - বুদ্ধিমান এজেন্ট তৈরি করার জন্য
- **Azure OpenAI** - LLM সক্ষমতার জন্য
- **A2A Protocol** - এজেন্ট যোগাযোগের মানক পদ্ধতির জন্য
- **Uvicorn** - স্থানীয় A2A সার্ভার চালানোর জন্য

### আপনি যা শিখেছেন:
1. **এজেন্ট তৈরি**: Semantic Kernel ব্যবহার করে বিশেষায়িত এজেন্ট তৈরি করা
2. **A2A ইন্টিগ্রেশন**: SK এজেন্টকে A2A প্রোটোকলের সাথে সামঞ্জস্যপূর্ণ করা
3. **এজেন্ট অর্কেস্ট্রেশন**: একাধিক বিশেষজ্ঞদের সমন্বয় করার জন্য ম্যানেজার এজেন্ট ব্যবহার করা
4. **রিয়েল-টাইম সার্ভিস**: বাইরের API (মুদ্রার হার জন্য Frankfurter) ইন্টিগ্রেশন করা
5. **স্থানীয় ডিপ্লয়মেন্ট**: একটি নোটবুকে একাধিক A2A সার্ভার চালানো

### পরবর্তী পদক্ষেপ:
- এজেন্টগুলোকে Azure Container Instances বা Azure Functions-এ ডিপ্লয় করুন
- আরও বিশেষায়িত এজেন্ট যোগ করুন (ফ্লাইট বুকিং, হোটেল সুপারিশ)
- এজেন্ট মেমোরি প্রয়োগ করুন, যাতে প্রসঙ্গ-সচেতন কথোপকথন সম্ভব হয়
- প্রোডাকশন ডিপ্লয়মেন্টের জন্য অথেনটিকেশন এবং সিকিউরিটি যোগ করুন
- ট্রাভেল প্ল্যানিং সিস্টেমের জন্য একটি ওয়েব ইন্টারফেস তৈরি করুন

### এই আর্কিটেকচারের প্রধান সুবিধা:
- **মডুলারিটি**: প্রতিটি এজেন্ট আলাদাভাবে তৈরি এবং ডিপ্লয় করা যায়
- **স্কেলেবিলিটি**: চাহিদা অনুযায়ী এজেন্টগুলো স্কেল করা যায়
- **পুনঃব্যবহারযোগ্যতা**: এজেন্টগুলো বিভিন্ন অ্যাপ্লিকেশনে পুনঃব্যবহার করা যায়
- **ইন্টারঅপারেবিলিটি**: A2A প্রোটোকল বিভিন্ন ফ্রেমওয়ার্কের এজেন্টের সাথে ইন্টিগ্রেশন সম্ভব করে



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিক অনুবাদ প্রদানের চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা তার জন্য দায়বদ্ধ থাকব না।
